In [1]:
%pwd

'/Users/neehanth/Documents/Data Scientist/Machine Learning/chicken_disease_clf/research'

In [2]:
import os

os.chdir('../')
%pwd

'/Users/neehanth/Documents/Data Scientist/Machine Learning/chicken_disease_clf'

In [3]:
import tensorflow as tf

model = tf.keras.models.load_model('artifacts/training/trained_model.h5')

/Users/neehanth/Documents/Data Scientist/Machine Learning/chicken_disease_clf/cnn_venv/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):
2025-12-22 20:57:14.052271: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-12-22 20:57:14.052292: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-12-22 20:57:14.052297: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.66 GB
I0000 00:00:1766458634.052475  258468 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1766458634.052542  258468 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: 

#### Evaluation Entity

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class EvaluationConfig:
    model_path: Path
    training_data: Path
    params: dict
    params_image_size: list
    params_batch_size: int

#### Configuration Manager - Evaluation

In [5]:
from cnn_classifier.constants import *
from cnn_classifier.utils.utilities import read_yaml, create_directories, save_json

In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        evaluation_config = EvaluationConfig(
            model_path='artifacts/training/trained_model.h5',
            training_data='artifacts/data_ingestion/data/images',
            params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )

        return evaluation_config

#### Evaluation Component

In [10]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.valid_generator = None
    
    def _validation_generator(self):
        validation_ds = tf.keras.utils.image_dataset_from_directory(
            subset="validation",
            shuffle=False,
            directory=self.config.training_data,
            validation_split=0.20,
            seed=42,
            image_size=self.config.params_image_size[:-1],  # e.g., [224, 224]
            batch_size=self.config.params_batch_size,
            label_mode="categorical",
            interpolation="bilinear"
        )

        self.class_names = validation_ds.class_names

        AUTOTUNE = tf.data.AUTOTUNE
        rescale = tf.keras.layers.Rescaling(1.0 / 255)

        validation_ds = validation_ds.map(
            lambda x, y: (rescale(x), y),
            num_parallel_calls=AUTOTUNE
        )
        validation_ds = validation_ds.cache().prefetch(AUTOTUNE)
        self.valid_generator = validation_ds
    
    def evaluation(self):
        self.model = self.load_model(self.config.model_path)
        self._validation_generator()
        self.score = self.model.evaluate(self.valid_generator)
    
    def save_score(self):
        scores = scores = {
            "loss": float(self.score[0]),
            "accuracy": float(self.score[1]),
            "class_names": list(self.class_names)
        }
        save_json(path=Path("scores.json"), data=scores)
    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

#### Evaluation Pipeline

In [11]:
try:
    config = ConfigurationManager()
    evaluation_config = config.get_evaluation_config()
    evaluation = Evaluation(evaluation_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
   raise e

Found 8067 files belonging to 4 classes.
Using 1613 files for validation.
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 387ms/step - accuracy: 0.9448 - loss: 0.4696
